# Tinyflux: Example with fictious temperature readings created through an LLM

In [2]:
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery, MeasurementQuery
from datetime import datetime, timezone, timedelta

## Initialize the TinyFlux database

In [3]:
db = TinyFlux("random_temperature_Data.db")

## Writing a CSV file into Tinyflux (directly, without exporting to a Python DataFrame first)

In [5]:
# Read CSV and insert data into TinyFlux
with open("example_data/tinyflux_time_series_dataset_with_tags.csv", mode="r") as file:
    ###  more="r" steht für "read-only mode"
    reader = csv.DictReader(file)
    for row in reader:
    ### hier wird tatsächlich in einem Loop jede Zeile einzeln abgehandelt, deswegen dauert das auch so lange    
        px = Point(
            time=datetime.strptime(row["timestamp"], "%Y-%m-%d %H:%M:%S"),
            ### this is a specific python method und steht für "string parse time" (parse time saved as string to time)
            ### das "%" im Zeitformat Platzhalter dient als "padding", quasi für leading-zero 
            measurement="weather",
            fields={"temperature": float(row["temperature"])},  # Only numeric fields
            tags={
                ### numerical data is not supported in tags, for that a second measurement would need to be created
                "condition": row["weather_condition"],  # Move non-numeric fields to tags
                "landscape_type": row["location"]
            }
        )
        db.insert(px)

In [25]:
### test des Inserts oben, gem. Übungsaufgabe sollen die ersten 10 Zeilen ausgegeben werden. TinyFlux scheint
### aber keine Methode zu haben, die Query auf N Einträge zu limitieren

measurement = MeasurementQuery()
results = db.search(measurement == "weather")

# Get the first N entries in the "results" array
for x in results[:10]:
    print(x)


Point(time=2020-01-01T00:00:00+00:00, measurement=weather, tags=condition:rainy; landscape_type:rural, fields=temperature:22.483570765056164)
Point(time=2020-01-01T00:00:00+00:00, measurement=weather, tags=condition:rainy; landscape_type:rural, fields=temperature:22.483570765056164)
Point(time=2020-01-01T01:00:00+00:00, measurement=weather, tags=condition:stormy; landscape_type:suburb, fields=temperature:19.308678494144075)
Point(time=2020-01-01T01:00:00+00:00, measurement=weather, tags=condition:stormy; landscape_type:suburb, fields=temperature:19.308678494144075)
Point(time=2020-01-01T02:00:00+00:00, measurement=weather, tags=condition:clear; landscape_type:suburb, fields=temperature:23.23844269050346)
Point(time=2020-01-01T02:00:00+00:00, measurement=weather, tags=condition:clear; landscape_type:suburb, fields=temperature:23.23844269050346)
Point(time=2020-01-01T03:00:00+00:00, measurement=weather, tags=condition:stormy; landscape_type:rural, fields=temperature:27.61514928204013)
Po

## Writing manually into Tinyflux

In [8]:
# Create manual records
### the measurements are missing, all these new entry are recorded but with measurement "default"
### because this codeblock ran multuple times, data data is duplicated. Should the "primary key" not have prevented this?

p1 = Point(
    time=datetime(2024, 9, 29, 15, 0, tzinfo=timezone.utc),
    tags={"landscape_type": "suburb", "condition": "overcast"},
    fields={"temperature": 16.5}
)

p2 = Point(
    time=datetime(2024, 9, 29, 16, 0, tzinfo=timezone.utc),
    tags={"landscape_type": "suburb", "condition": "rainy"},
    fields={"temperature": 17.3}
)

p3 = Point(
    time=datetime(2024, 9, 29, 14, 0, tzinfo=timezone.utc),
    tags={"landscape_type": "suburb", "condition": "rainy"},
    fields={"temperature": 15.7}
)


p4 = Point(
    time=datetime(2024, 9, 29, 13, 0, tzinfo=timezone.utc),
    tags={"landscape_type": "suburb", "condition": "rainy"},
    fields={"temperature": 14.9}
)

p5 = Point(
    time=datetime(2024, 9, 29, 11, 0, tzinfo=timezone.utc),
    tags={"landscape_type": "suburb", "condition": "cloudy"},
    fields={"temperature": 13.8}
)
### in the previous block it was part of a loop, here it could be done with an array:
### db.insert([p1,p2])
db.insert(p1)
db.insert(p2)
db.insert(p3)
db.insert(p4)
db.insert(p5)   


1

## Querying by Time

In [9]:
# Search for a time value
Time = TimeQuery()
time_start = Time >= datetime(2024, 1, 1, tzinfo=timezone.utc)
time_end = Time < datetime(2024, 12, 31, tzinfo=timezone.utc)
time_count = db.count(time_start & time_end)
print("Time Query Count:")
print(time_count)

### thats correct, the dataset is from 2020 and I only have 2 measurements from 2024

Time Query Count:
11


## Querying by Field

In [28]:
# Search for a field value
### I do not understand these 2 lines at all
Field = FieldQuery()
field_results = db.select("fields.temperature", Field.temperature > 15.00)
print("\nField Query Results:")
### the \n designates a new line
for x in field_results:
    print(x)


Field Query Results:
22.483570765056164
19.308678494144075
23.23844269050346
27.61514928204013
18.829233126383322
18.8293152152541
27.896064077536955
23.837173645764544
17.65262807032524
22.712800217929825
17.682911535937688
17.671351232148716
21.20981135783017
17.188562353795135
21.571236662976368
15.459879622393945
27.32824384460777
18.871118497567323
20.33764102343962
17.278086377374088
20.55461294854933
21.87849009172836
16.996806550405974
18.541531251033618
16.991466938853016
29.261390922544688
19.93251387631033
24.112724560515943
21.044317975023777
20.984306179345616
23.692332899977053
20.85684140594985
19.421758588058797
18.494481522053555
16.400778958026457
17.696806145201062
25.285611131094576
21.71809144784231
21.620419846973974
18.074588597918417
16.615389998470206
23.05838144420434
25.154997612479754
24.656400595580994
15.803912383886807
18.453938120743928
21.65631715701782
24.877725635611796
17.60412881077355
19.071705116680914
24.062629111970992
26.781200142854114
19.639

In [ ]:
### bonus from the slides: Try to use an aggregate function like sum or avg on a field
### I dont think TinyFlux supports aggregate functions...you need to convert points results list into a DF first



## Querying by Tag

In [4]:
# Search for a tag value
Tag = TagQuery()
tag_results = db.search(Tag.landscape_type == 'suburb')
print("\nTag Query Results:")
for x in tag_results:
    print(x)

### how would you approach this if the tag was not unique but existed in multiple measurements within the same database?    


Tag Query Results:
Point(time=2020-01-01T01:00:00+00:00, measurement=weather, tags=condition:stormy; landscape_type:suburb, fields=temperature:19.308678494144075)
Point(time=2020-01-01T01:00:00+00:00, measurement=weather, tags=condition:stormy; landscape_type:suburb, fields=temperature:19.308678494144075)
Point(time=2020-01-01T02:00:00+00:00, measurement=weather, tags=condition:clear; landscape_type:suburb, fields=temperature:23.23844269050346)
Point(time=2020-01-01T02:00:00+00:00, measurement=weather, tags=condition:clear; landscape_type:suburb, fields=temperature:23.23844269050346)
Point(time=2020-01-01T12:00:00+00:00, measurement=weather, tags=condition:rainy; landscape_type:suburb, fields=temperature:21.20981135783017)
Point(time=2020-01-01T12:00:00+00:00, measurement=weather, tags=condition:rainy; landscape_type:suburb, fields=temperature:21.20981135783017)
Point(time=2020-01-01T14:00:00+00:00, measurement=weather, tags=condition:stormy; landscape_type:suburb, fields=temperature:

In [ ]:
###  bonus from the slides: use a combination of grouping-by with an aggregate, like eg. count
### I dont think TinFlux supports  aggregate functions or even groupings...


In [9]:
### super bonus: how could you combine queries that have limitations on tags, field, and/or measurement?

Time = TimeQuery()
Tag = TagQuery()
q1 = Time > datetime(2024, 1, 1, tzinfo=timezone.utc)
q2 = Tag.condition == "cloudy"
qx = db.search(q1 & q2)

for x in qx:
    print(x)

Point(time=2024-09-29T11:00:00+00:00, measurement=_default, tags=landscape_type:suburb; condition:cloudy, fields=temperature:13.8)
